In [2]:
import sys, subprocess
print("Python:", sys.version)
print("Executable:", sys.executable)

subprocess.check_call([sys.executable, "-m", "pip", "install", "-U", "onnx2tf"])


Python: 3.10.7 (v3.10.7:6cc6b13308, Sep  5 2022, 14:02:52) [Clang 13.0.0 (clang-1300.0.29.30)]
Executable: /Library/Frameworks/Python.framework/Versions/3.10/bin/python3



[notice] A new release of pip is available: 23.3.1 -> 25.3
[notice] To update, run: /Library/Frameworks/Python.framework/Versions/3.10/bin/python3 -m pip install --upgrade pip


0

In [3]:
import sys, subprocess
print("Python:", sys.version)
print("Executable:", sys.executable)

subprocess.check_call([
    sys.executable, "-m", "pip", "install", "-U", "tf_keras<=2.19.0"
])


Python: 3.10.7 (v3.10.7:6cc6b13308, Sep  5 2022, 14:02:52) [Clang 13.0.0 (clang-1300.0.29.30)]
Executable: /Library/Frameworks/Python.framework/Versions/3.10/bin/python3



[notice] A new release of pip is available: 23.3.1 -> 25.3
[notice] To update, run: /Library/Frameworks/Python.framework/Versions/3.10/bin/python3 -m pip install --upgrade pip


0

In [4]:
from ultralytics import YOLO
import os, shutil, random
from PIL import Image
import numpy as np

In [3]:
source_dir = "./us_b"

# Define source and target directories
target_dir = "./ultrasound_data"  # our working directory for processed data
classes = ["benign", "malignant", "normal"]

# Create train/val directories
for split in ["train", "val"]:
    for cls in classes:
        os.makedirs(os.path.join(target_dir, split, cls), exist_ok=True)

# Gather all image file paths, excluding mask files
data_images = {cls: [] for cls in classes}
for cls in classes:
    cls_dir = os.path.join(source_dir, cls)
    # Some images might be in subfolders or have mask files; handle accordingly
    for root, _, files in os.walk(cls_dir):
        for fname in files:
            if "mask" in fname.lower():
                continue  # skip mask images
            if fname.lower().endswith(('.png', '.jpg', '.jpeg')):  # image file
                data_images[cls].append(os.path.join(root, fname))

# Shuffle and split each class
split_ratio = 0.2  # 20% val
train_files = {cls: [] for cls in classes}
val_files = {cls: [] for cls in classes}
random.seed(42)
for cls, files in data_images.items():
    random.shuffle(files)
    n_val = int(len(files) * split_ratio)
    val_files[cls] = files[:n_val]
    train_files[cls] = files[n_val:]
    print(f"{cls.capitalize()}: {len(train_files[cls])} train, {len(val_files[cls])} val images")

# Copy images to target folders with conversion to 3-channel
for cls in classes:
    for fpath in train_files[cls]:
        img = Image.open(fpath)
        img_rgb = img.convert("RGB")  # ensure 3-channel
        # Save to train folder, preserving filename
        fname = os.path.basename(fpath)
        img_rgb.save(os.path.join(target_dir, "train", cls, fname))
    for fpath in val_files[cls]:
        img = Image.open(fpath)
        img_rgb = img.convert("RGB")
        fname = os.path.basename(fpath)
        img_rgb.save(os.path.join(target_dir, "val", cls, fname))


Benign: 351 train, 87 val images
Malignant: 169 train, 42 val images
Normal: 108 train, 26 val images


In [4]:

# Load a YOLOv8 nano classification model, pretrained on ImageNet
model = YOLO('yolov8n-cls.pt')  # this will download the weights if not present

# Train the model on our dataset
results = model.train(
    data=target_dir,   # path to dataset root (with train/ and val/ subfolders)
    epochs=50,
    imgsz=224,
    batch=32,
    verbose=True  # print training progress
)


New https://pypi.org/project/ultralytics/8.3.228 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.225 🚀 Python-3.10.7 torch-2.7.0 CPU (Apple M1)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./ultrasound_data, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train5, nbs=64, nms=False, opset=None, optimize=False, optimizer

In [5]:
# Load the best model from training
best_model = YOLO('runs/classify/train/weights/best.pt')

# Validate on the validation set to get metrics
metrics = best_model.val(data=target_dir, split='val', imgsz=224)
print(metrics)  # this will show overall accuracy and per-class accuracy if available


Ultralytics 8.3.225 🚀 Python-3.10.7 torch-2.7.0 CPU (Apple M1)
YOLOv8n-cls summary (fused): 30 layers, 1,438,723 parameters, 0 gradients, 3.3 GFLOPs
train: /Users/graysonrichard/Desktop/cs_2025/ultrasound/ultrasound_data/train... found 628 images in 3 classes ✅ 
val: /Users/graysonrichard/Desktop/cs_2025/ultrasound/ultrasound_data/val... found 155 images in 3 classes ✅ 
test: None...
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1529.6±1308.0 MB/s, size: 229.2 KB)
val: Scanning /Users/graysonrichard/Desktop/cs_2025/ultrasound/ultrasound_data/val... 155 images, 0 corrupt: 100% ━━━━━━━━━━━━ 155/155 549.5Kit/s 0.0s
               classes   top1_acc   top5_acc: 100% ━━━━━━━━━━━━ 10/10 3.7it/s 2.7s.3ss
                   all      0.903          1
Speed: 0.0ms preprocess, 10.3ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to /Users/graysonrichard/Desktop/cs_2025/ultrasound/runs/classify/val4
ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion

In [6]:
!which python3
!python3 --version
!python3 -m pip install --upgrade onnx onnxruntime onnxslim onnx_graphsurgeon ai-edge-litert tf_keras tensorflow


/Library/Frameworks/Python.framework/Versions/3.11/bin/python3
Python 3.11.1
  Using cached ai_edge_litert-2.0.3-cp311-cp311-macosx_12_0_arm64.whl.metadata (1.9 kB)
  Using cached tf_keras-2.20.1-py3-none-any.whl.metadata (1.8 kB)
  Using cached tensorflow-2.20.0-cp311-cp311-macosx_12_0_arm64.whl.metadata (4.5 kB)
  Using cached tensorboard-2.20.0-py3-none-any.whl.metadata (1.8 kB)
Using cached ai_edge_litert-2.0.3-cp311-cp311-macosx_12_0_arm64.whl (179 kB)
Using cached tf_keras-2.20.1-py3-none-any.whl (1.7 MB)
Using cached tensorflow-2.20.0-cp311-cp311-macosx_12_0_arm64.whl (200.5 MB)
Using cached tensorboard-2.20.0-py3-none-any.whl (5.5 MB)
  Attempting uninstall: ai-edge-litert
    Found existing installation: ai-edge-litert 1.3.0
    Uninstalling ai-edge-litert-1.3.0:
      Successfully uninstalled ai-edge-litert-1.3.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.19.0
    Uninstalling tensorboard-2.19.0:
      Successfully uninstalled tensorboa

In [7]:
# Export the best model to TFLite with int8 quantization
best_model.export(format='tflite', int8=True, data=target_dir, imgsz=224)


Ultralytics 8.3.225 🚀 Python-3.10.7 torch-2.7.0 CPU (Apple M1)

PyTorch: starting from 'runs/classify/train/weights/best.pt' with input shape (1, 3, 224, 224) BCHW and output shape(s) (1, 3) (2.8 MB)
requirements: Ultralytics requirements ['tf_keras<=2.19.0', 'sng4onnx>=1.0.1', 'onnx_graphsurgeon>=0.3.26', 'ai-edge-litert>=1.2.0,<1.4.0', 'onnx2tf>=1.26.3', 'protobuf>=5'] not found, attempting AutoUpdate...
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/1.7 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 23.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/5.1 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 24.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/252.6 MB ? eta -:--:--
   ━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/252.6 MB 43.8 MB/s eta 0:00:06
   ━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/252.6 MB 38.0 MB/s eta 0:00:07
 

ModuleNotFoundError: No module named 'onnx2tf'